# **Ranking Long Snappers (LS) from Special Teams with LOAFT Score**

In [ ]:
from IPython.display import Image
Image("../input/loaft-image/logo.png", embed = True, retina = True)

### **What is a Long Snapper (LS)?**
A Long Snapper (LS) is the player on a Special Team whose duty is to snap the football over a longer distance, typically around 15 yards during punts, and 7–8 yards during field goals and extra point attempts. During field goal and point after touchdown attempts, the snap is received by the holder, typically 7–8 yards away. During punts, the snap is delivered to the punter from 13–15 yards away. Following a punt snap, the snapper often executes a blocking assignment and then must cover the kick by running downfield and attempting to stop the opposing team's punt returner from advancing the ball in the opposite direction. If the punt goes uncaught, it is the snapper's responsibility to make sure the ball does not enter the end zone or bounce backward resulting in loss of yards. The majority of snappers at the highest levels of competition are specialized, meaning that they uniquely play the position of snapper, or have limited responsibilities elsewhere.  
The position of long snapper was not always a solid position. Until about 20 years ago, the position of long snapper would be filled by a random player or lineman who was not getting playing time. The thought to have a roster spot reserved for just a snapper was ridiculous. Over time, people started to realize how important the role was, as one bad snap could lose any game. In the past 20 years, teams have not only trained and recruited true long snappers, but they even offer scholarships because of the importance of the position.  
Overall, Long Snappers (LS) need to be fast with the ability to accelerate and change directions quickly while also being tall and heavy to block strong Players from the opposite team. Therefore, a ranking among Long Snappers (LS) should consider mainly physical/fitness aspects of their performance.

### **What are the relevant attributes a Long Snapper (LS) can be juged by?**
The hereby introduced Long Snapper Adjusted Fitness Total (LOAFT) score takes into consideration the following data:
1. From the Player data:
    - Height
    - Weight
    - Birth Date
    - Position
    - NFL-ID
2. From the Tracking data:
    - Speed in yards/second
    - Acceleration in yards/second^2
    - Distance traveled from prior time point (in yards)
    
Information are filtered in such a way that only Players with the position "Long Snapper" are taken into consideration. The NFL-ID is needed to uniquly identify a single Long Snapper (LS).

**Note: Other statistics from Play data or Scouting data is not taken into consideration since statistics/achievements concerning the entire Special Team cannot always be clearly assigned to a single Player, making it hard to keep a ranking objective. This underlines the fact that the LOAFT statistics are meant to specificly probe Long Snappers' Physical Fitness - not their overall behavior or integration into the game. Therefore, LOAFT statistics can help analysts to pick out Long Snappers by their velocity relativ to their height and weight or to find patterns in the way physical attributes influence Agility and Maneuverability, they cannot replace but rather supplement overall statistics.**

### **Agenda**
#### 1. Loading, Analyzing & Visualizing relevant data
#### 2. Building dataframes required for Comparison
#### 3. Developing statistical functions & Ranking Long Snappers (LS)


**Note: The entire Analytics Circle will be shown with Tracking data from 2018**

### **Loading & Visualizing relevant data**

#### Tracking data

In [ ]:
import pandas as pd

# Loading required .csv file
df_tracking18 = pd.read_csv("../input/nfl-big-data-bowl-2022/tracking2018.csv", sep = r',', skipinitialspace = True)

# Only taking Kick Returnes (KR) into consideration
df_tracking18 = df_tracking18[df_tracking18["position"] == "LS"]

# Pre-Cleaning data by dropping irrelevant columns
df_tracking18 = df_tracking18.drop(["time", "jerseyNumber", "event", "playDirection", "playId", "displayName", "team", "x", "y", "frameId", "o"], axis = 1)

df_tracking18

#### How many different NFL Players are there in 2018 who play as Long Snapper (LS)?

In [ ]:
print(df_tracking18['nflId'].nunique())

#### What is the average speed/ acceleration/ distance traveled in a Tracking Frame Scene from a Long Snapper (LS)?

In [ ]:
avg_speed = df_tracking18["s"].mean()
avg_acc = df_tracking18["a"].mean()
avg_dis = df_tracking18["dis"].mean()

print("Average Speed: " + str(avg_speed) + " yards/s")
print("Average Acceleration: " + str(avg_acc) + " yards/s^2")
print("Average Distance from prior Time Point: " + str(avg_dis) + " yards")

#### Players data

In [ ]:
import pandas as pd

df_players = pd.read_csv("../input/nfl-big-data-bowl-2022/players.csv", sep = r',', skipinitialspace = True)
df_players = df_players.drop(["collegeName", "Position", "displayName"], axis = 1)
df_players["height"] = df_players["height"].str.replace("-", "")
df_players['birthDate'] = df_players['birthDate'].str.split('-').str[0]

df_players

### **Building dataframes required for Comparison**

#### Merging Tracking data and Players data

In [ ]:
df = pd.merge(df_tracking18, df_players, on = "nflId", how = "outer")

# Converting columns to numeric
df["weight"] = pd.to_numeric(df["weight"])
df["height"] = pd.to_numeric(df["height"])

df.head()

### **Developing statistical functions & Ranking Long Snappers (LS)**

#### Defining LOAFT function

The LOAFT function calculates the score in the following way:  
1. The fitness related metrics speed, acceleration and distance are averaged for the entire year/season, assigning the player a certain fitness score
2. Then, the player's height and weight are taken into consideration to adjust the score. Since it it harder for taller and heavier players to move quickly and agile and since additional weight is an advantage when it comes to blocking, heavier/taller players score higher than lighter/smaller players with the same fitness score.

All that needs to be done is calling the function **LOAFT()** with the nlfID as the parameter **id**. The output represents the Long Snapper Adjusted Fitness Total (LOAFT) score, which can be printed and/or assigned to a variable.

In [ ]:
def LOAFT(id):
    # Picking player from dataframe according to id (nflId)
    player = df[df["nflId"] == id]
    
    # Calculation average metrics
    avg_speed_player = player["s"].mean()
    avg_acc_player = player["a"].mean()
    avg_dis_player = player["dis"].mean()
    
    # Calculating fitness factor
    fitness_factor = avg_speed_player + avg_acc_player + avg_dis_player
    
    # Calculation LOAFT Score
    player_height = player["height"].values[0]
    height_correction = 0.01 * player_height

    player_weight = player["weight"].values[0]
    weight_correction = 0.01 * player_weight

    loaft_score = fitness_factor + height_correction + weight_correction
    
    return loaft_score

#### Applying LOAFT function

In [ ]:
loaft = LOAFT(36086.0)
loaft

In [ ]:
loaft = LOAFT(39750.0)
loaft

#### Comparing Long Snappers (LS) to each other

In [ ]:
# Show nflIds of unique Long Snappers in the dataset
list = df_tracking18['nflId'].unique()
print(list)

In [ ]:
for i in list:
    loaft_scores = []
    loaft_scores.append(LOAFT(i))
    print(loaft_scores)

The LOAFT function is not restricted to years/seasons as shown here. Long Snappers' LOAFT scores can also be calculated for each game, week or even an entire team. Also, comparisons can be made more detailed, if Players are only compared when having the same age, playing in the same climate or participating in similar tactics.

**Feel free to use and apply the LOAFT score or to visualize LOAFT score comparisons in charts and graphs...**